In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm

In [9]:
dataset_dir = '/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/'

train_list_file = dataset_dir + 'train_feature_split1.txt'
train_lines = open(train_list_file).readlines() 
train_lines = [x.strip() for x in train_lines]
print(train_lines[:10])

val_list_file = dataset_dir + 'val_feature_split1.txt'
val_lines = open(val_list_file).readlines() 
val_lines = [x.strip() for x in val_lines]
print(val_lines[:10])



['train/non-covid/subset2/ct_scan_418 non-covid 21 44 0 0 512 438', 'train/non-covid/subset2/ct_scan_429 non-covid 26 47 0 99 497 425', 'train/non-covid/subset4/ct_scan_701 non-covid 22 46 10 21 487 378', 'train/non-covid/subset4/ct_scan_627 non-covid 48 75 27 71 485 426', 'train/non-covid/subset5/ct_scan985 non-covid 63 174 16 74 512 397', 'train/non-covid/subset3/ct_scan_3 non-covid 53 142 0 0 512 478', 'train/non-covid/subset4/ct_scan_686 non-covid 30 84 0 86 497 419', 'train/non-covid/subset3/ct_scan_183 non-covid 135 267 0 119 512 435', 'train/non-covid/subset4/ct_scan_706 non-covid 40 88 51 67 503 413', 'train/non-covid/subset3/ct_scan_105 non-covid 13 30 0 109 512 445']
['validation/non-covid/non-covid/ct_scan_21 non-covid 111 308 0 23 512 442', 'validation/non-covid/val_non_covid/ct_scan214 non-covid 154 313 0 84 512 512', 'validation/non-covid/non-covid/ct_scan_161 non-covid 26 68 7 104 507 443', 'validation/non-covid/non-covid/ct_scan_134 non-covid 31 73 32 89 508 417', 'vali

In [10]:
covid_dir = '/media/ubuntu/MyHDataStor3/datasets/COV19D/'

train_annot_file = covid_dir + 'train_partition_covid_categories.csv' 
train_annot = open(train_annot_file,'r').readlines() 
train_annot = train_annot[1:]
train_annot = [x.strip() for x in train_annot]
print(train_annot[:10])

val_annot_file = covid_dir + 'val_partition_covid_categories.csv' 
val_annot = open(val_annot_file,'r').readlines() 
val_annot = val_annot[1:]
val_annot = [x.strip() for x in val_annot]
print(val_annot[:10])


['ct_scan_1;4', 'ct_scan_5;1', 'ct_scan_6;3', 'ct_scan_7;3', 'ct_scan_9;2', 'ct_scan_11;3', 'ct_scan_12;2', 'ct_scan_15;1', 'ct_scan_16;1', 'ct_scan_18;1']
['ct_scan_0;2', 'ct_scan_5;1', 'ct_scan_7;3', 'ct_scan_13;3', 'ct_scan_14;3', 'ct_scan_16;2', 'ct_scan_19;4', 'ct_scan_21;1', 'ct_scan_25;3', 'ct_scan_26;1']


In [22]:
train_lines_new = [] 
for a in train_annot:
    #print(a)
    a_tmp = a.split(';')
    scan = a_tmp[0]
    cls = a_tmp[1]
    #print(scan,cls)
    lines = [x for x in train_lines if scan==x.split(' ')[0].split('/')[-1] and x.split(' ')[1]=='covid'][0]
    train_lines_new.append((lines,cls))

print(train_lines_new[:10],len(train_lines_new))    

[('train/covid/subset1/ct_scan_1 covid 89 206 0 100 512 478', '4'), ('train/covid/subset1/ct_scan_5 covid 113 308 0 55 512 442', '1'), ('train/covid/subset1/ct_scan_6 covid 17 42 21 149 461 425', '3'), ('train/covid/subset1/ct_scan_7 covid 129 225 0 108 491 438', '3'), ('train/covid/subset1/ct_scan_9 covid 123 185 0 124 512 489', '2'), ('train/covid/subset1/ct_scan_11 covid 121 168 0 90 512 466', '3'), ('train/covid/subset1/ct_scan_12 covid 116 236 0 82 512 476', '2'), ('train/covid/subset1/ct_scan_15 covid 105 261 5 49 512 423', '1'), ('train/covid/subset1/ct_scan_16 covid 15 33 0 107 512 451', '1'), ('train/covid/subset1/ct_scan_18 covid 116 237 0 100 512 444', '1')] 258


In [23]:
val_lines_new = [] 
for a in val_annot:
    #print(a)
    a_tmp = a.split(';')
    scan = a_tmp[0]
    cls = a_tmp[1]
    #print(scan,cls)
    lines = [x for x in val_lines if scan==x.split(' ')[0].split('/')[-1] and x.split(' ')[1]=='covid'][0]
    val_lines_new.append((lines,cls))

print(val_lines_new[:10],len(val_lines_new))    

[('validation/covid/covid/ct_scan_0 covid 112 189 0 89 512 441', '2'), ('validation/covid/covid/ct_scan_5 covid 129 249 0 115 512 492', '1'), ('validation/covid/covid/ct_scan_7 covid 18 42 39 63 512 463', '3'), ('validation/covid/covid/ct_scan_13 covid 71 188 0 88 512 461', '3'), ('validation/covid/covid/ct_scan_14 covid 70 86 46 128 435 355', '3'), ('validation/covid/covid/ct_scan_16 covid 109 261 0 90 512 439', '2'), ('validation/covid/covid/ct_scan_19 covid 103 213 8 83 479 423', '4'), ('validation/covid/covid/ct_scan_21 covid 129 219 0 75 512 454', '1'), ('validation/covid/covid/ct_scan_25 covid 92 196 30 104 512 460', '3'), ('validation/covid/covid/ct_scan_26 covid 111 273 26 75 490 400', '1')] 61


In [24]:
split =2 
train_split_file = dataset_dir + 'train_feature_split{}.txt'.format(split)

fp = open(train_split_file,'w') 
for line in train_lines_new:
    l = line[0].split(' ')
    cls = line[1]
    fp.write("{} {} {} {} {} {} {} {}\n".format(l[0],cls,l[2],l[3],l[4],l[5],l[6],l[7]))
fp.close()

In [25]:
val_split_file = dataset_dir + 'val_feature_split{}.txt'.format(split)
fp = open(val_split_file,'w') 
for line in val_lines_new:
    l = line[0].split(' ')
    cls = line[1]
    fp.write("{} {} {} {} {} {} {} {}\n".format(l[0],cls,l[2],l[3],l[4],l[5],l[6],l[7]))
fp.close()